# Comparing the means of two distributions

Load the data. This is part of the INSARK-data that we have worked with before. It was made public by the national Swedish archives.  

In [ ]:
options(repr.plot.width=14, repr.plot.height=8)
suppressMessages(require(dplyr))
suppressMessages(require(ggplot2))
data <- readRDS("insark_h_w_born.rds")
names(data)
table(data$born)

Extract the data for the different periods and define a "t-value" function

In [ ]:
dat1 <- data %>% filter(born=="51-54")
dat2 <- data %>% filter(born=="55-59")
## function that computes a t-value for a given sample size
my_t_test <- function(samp1,samp2){
    ## sample at random from d1 and d2 with size N and compute the t-statistic
    N<- length(samp1)
    m1 <- mean(samp1)
    m2 <- mean(samp2)
    v1 <- var(samp1)
    v2 <- var(samp2)
    sp <- sqrt((v1+v2)/2)
    t <- (m2-m1)/(sp/sqrt(N/2))
    return(t)
}
## calculate the "population" means as the average of the full data
m1 <- mean(dat1$w)
m2 <- mean(dat2$w)
print(paste("mean weight population 1:",round(m1,1)))
print(paste("mean weight population 2:",round(m2,1)))

Next we sample from these populations and do a t-test to investigate of the means differ. 

In [ ]:
nrep <- 1000
nsamp <- 10
tval <- vector(length=nrep)
for (i in 1:nrep){
    samp1 <- sample(dat1$w,nsamp)
    samp2 <- sample(dat2$w,nsamp)
    tval[i] <- my_t_test(samp1,samp2)
}
hist(tval)

Instead of just plotting the t-values we will now do a formal t-test. For this we need to know the critical value that the sample $t$ value has to exceeed for us to reject the null-hypothesis.

In [ ]:
## use a significance level of 0.01 (two sided)
alpha <- 0.01/2
df <- nsamp*2-2
tcrit <- qt(1-alpha,df)

Now we can check if the $t$-value of the sample exceeds the critical value. Change the sample size to see how that influence the probability of rejecting $H_0$.

In [ ]:
nsamp <- 100
samp1 <- sample(dat1$w,nsamp)
samp2 <- sample(dat2$w,nsamp)
tval <- my_t_test(samp1,samp2)
print(paste("mean sample 1:",mean(samp1)))
print(paste("mean sample 2:",mean(samp2)))
print(paste("t-value:", tval)) 
print(paste("t-test:", ifelse(abs(tval) > tcrit,"reject H0","accept H0")))

We can do this more systematically by repeating this many times and calculating how often we reject $H_0$.In the code below we let "1" denote rejection and "0" acceptance. By summing the output and dividing by the total number of trials (nrep) we estimate the probability of rejecting $H_0$ when it is true. This is called the _power_ of the test. 

In [ ]:
nrep <- 500
nsamp <- 10
test_res <- vector(length=nrep)
for (i in 1:nrep){
    samp1 <- sample(dat1$w,nsamp)
    samp2 <- sample(dat2$w,nsamp)
    tval <- my_t_test(samp1,samp2)
    test_res[i] <- ifelse(abs(tval) > tcrit,1,0)
}
print(paste("test power", sum(test_res)/nrep))

**Question:** What is the sample size needed in order for the probability of rejecting $H_0$ exceeding 0.95? 